# MOVIE RECOMMENDATION USING AUTO ENCODERS

# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

# Constants

In [2]:
MOVIES_DAT_1M = 'Movielense_Dataset/ml-1m/movies.dat'
USERS_DAT_1M = 'Movielense_Dataset/ml-1m/users.dat'
RATINGS_DAT_1M = 'Movielense_Dataset/ml-1m/ratings.dat'
TRAINING_SET_100K_U1 = 'Movielense_Dataset/ml-100k/u1.base'
TEST_SET_100K_U1 = 'Movielense_Dataset/ml-100k/u1.test'

# Importing the Dataset

In [3]:
colnames = ['MOVIE_ID', 'MOVIE_NAME', 'GENERE']
movies = pd.read_csv(MOVIES_DAT_1M, sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = colnames)
movies.head()

,MOVIE_ID,MOVIE_NAME,GENERE
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
colnames = ['USER_ID', 'GENDER', 'AGE', 'USERS_JOB', 'ZIP_CODE']
users = pd.read_csv(USERS_DAT_1M, sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = colnames)
users.head()

,USER_ID,GENDER,AGE,USERS_JOB,ZIP_CODE
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
colnames = ['USER_ID', 'MOVIE_ID', 'RATINGS', 'TIMESTAMP']
ratings = pd.read_csv(RATINGS_DAT_1M, sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = colnames)
ratings.head()

,USER_ID,MOVIE_ID,RATINGS,TIMESTAMP
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Preparing the training set and the test set

In [6]:
colnames = ['USER_ID', 'MOVIE_ID', 'RATINGS', 'TIMESTAMP']
training_set = pd.read_csv(TRAINING_SET_100K_U1, delimiter = '\t', header = None, names = colnames)
training_set = np.array(training_set, dtype = 'int64') # converting it into numpy array
print('Training Set Shape: ', training_set.shape)

Training Set Shape:  (80000, 4)


In [7]:
colnames = ['USER_ID', 'MOVIE_ID', 'RATINGS', 'TIMESTAMP']
test_set = pd.read_csv(TEST_SET_100K_U1, delimiter = '\t', header = None, names = colnames)
test_set = np.array(test_set, dtype = 'int64') # converting it into numpy array
print('Test Set Shape: ', test_set.shape)

Test Set Shape:  (20000, 4)


# Getting Number of Users and Movies

In [8]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print('Number of Users: ', nb_users)
print('Number of Movies: ', nb_movies)

Number of Users:  943
Number of Movies:  1682


## Converting the data into an array with users in lines and movies in columns

In [9]:
"""converts the data into an array with users in lines and movies in columns"""
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [10]:
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors

In [11]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Creating the Architecture of the Neural Network

In [14]:
class SAE(nn.Module):
    """stacked auto encoders"""
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        """x - input vector"""
        x = self.activation(self.fc1(x)) # encoding
        x = self.activation(self.fc2(x)) # encoding
        x = self.activation(self.fc3(x)) # decoding
        x = self.fc4(x) # decoded output vector
        return x

In [15]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training the Stacked Auto Encoder

In [47]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = training_set[id_user].unsqueeze(0) # pytorch will only accept in batches.
                                                   # Hence with batch 1 (or) inline training
                                                   # Basically reshaping
        target = input.clone()
        if torch.sum(target.data > 0) > 0: # getting only the users who have rated atleast one movie
            output = sae.forward(input)
            target.require_grad = False # gradient should only be computed w.r.t input & not the target variable
                                        #  This will reduce the computations and saves up the memory
            output[target == 0] = 0 # These values will not count in computation of error
                                    # since these are the movies the user haven't rated yet
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10) # 1e-10 is added for safety,
                                                                                   # incase the denominator is zero
            loss.backward() # decrease the weights
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step() # decides the intensity of the weight to be decreased / increased
                             # decreased in this case as we've used loss.backward()
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

epoch: 1 loss: 1.7662807369421898
epoch: 2 loss: 1.096584254194618
epoch: 3 loss: 1.0533869730529886
epoch: 4 loss: 1.0382287552190683
epoch: 5 loss: 1.0309408815839998
epoch: 6 loss: 1.026459729473496
epoch: 7 loss: 1.023837112656753
epoch: 8 loss: 1.0217030243164513
epoch: 9 loss: 1.020562484715003
epoch: 10 loss: 1.019451787369806
epoch: 11 loss: 1.0189997818931271
epoch: 12 loss: 1.0184310270026153
epoch: 13 loss: 1.0177965263871165
epoch: 14 loss: 1.0175584351295701
epoch: 15 loss: 1.0173615050775942
epoch: 16 loss: 1.016970564676673
epoch: 17 loss: 1.0165178412788531
epoch: 18 loss: 1.0164072693303083
epoch: 19 loss: 1.016222020112096
epoch: 20 loss: 1.0163464047532178
epoch: 21 loss: 1.0160947316897966
epoch: 22 loss: 1.015964803044597
epoch: 23 loss: 1.0158801330878988
epoch: 24 loss: 1.0157779124955792
epoch: 25 loss: 1.0154018873375494
epoch: 26 loss: 1.0156976938572655
epoch: 27 loss: 1.0154120490236016
epoch: 28 loss: 1.0149982775805735
epoch: 29 loss: 1.012821639598742
epo

# Testing the SAE

In [66]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = training_set[id_user].unsqueeze(0) # pytorch will only accept in batches.
                                               # Hence with batch 1 (or) inline training
                                               # Basically reshaping
    target = test_set[id_user].unsqueeze(0)
    if torch.sum(target.data > 0) > 0: # getting only the users who have rated atleast one movie
        output = sae.forward(input)
        target.require_grad = False # gradient should only be computed w.r.t input & not the target variable
                                        #  This will reduce the computations and saves up the memory
        output[target == 0] = 0 # These values will not count in computation of error
                                # since these are the movies the user haven't rated yet
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10) # 1e-10 is added for safety,
                                                                               # incase the denominator is zero
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print('Test loss: ' + str(test_loss / s))

Test loss: 0.9523516666347843


**On average the predicted rating will be different from the real rating by less than 1 star**